In [139]:
# preps

import pandas as pd
import sqlite3

database_directory = "data/movies.db"

def get_data_from_db(database_directory, tablename):
    db = sqlite3.connect(database_directory)
    query = f"SELECT * FROM {tablename}"
    df = pd.read_sql(query, db)
    db.close()
    return df

In [254]:
# make searchable extract: keywords vs movieIds

def splitter1(x):
    return ",".join(x.split("|")).lower()
def splitter2(x):
    return ",".join(x.split(",")).lower()
def splitter3(x):
    return x.split(",")

# load data
dm = get_data_from_db(database_directory, "movielens")
dt = get_data_from_db(database_directory, "tags")

# make genres to a string
dm["genres"] = dm["genres"].apply(splitter1)

# group tags per movieid and make string out of it
dt = dt.groupby(["movieId"])['tag'].apply(lambda x: ','.join(x)).reset_index()
dt["tag"] = dt["tag"].apply(splitter2)

# merge combined tags on dm
df = pd.merge(dm, dt, how = "left", on = "movieId")
df["tag"].fillna(value = "", inplace = True)

# keyword column
df["keywords"] = df["genres"] + df["tag"]  + df["title"]

# make extact for searching
keywords = list(df["keywords"])
movieIds = list(df["movieId"])

keywords_and_movieids = list(zip(keywords, movieIds))

In [258]:
# Option1 : Match ALL criteria

filters = ["caprio", "action"]
filtered_ids = []
for i, movie in enumerate(keywords_and_movieids):
    found = []
    for keyword in filters:
        if keyword.lower() in keywords_and_movieids[i][0]:
            found.append(True)
        else:
            found.append(False)
    if all(found):
        filtered_ids.append(keywords_and_movieids[i][1])
    else:
        continue
filtered_ids = sorted(list(set(filtered_ids)))
filtered_ids

[49530, 79132, 99114]

In [257]:
# Option2: Match ANY criteria

filters = ["president"]
filtered_ids = []
for keyword in filters:
    for i, movie in enumerate(keywords_and_movieids):
        if keyword.lower() in keywords_and_movieids[i][0]:
            filtered_ids.append(keywords_and_movieids[i][1])
        else:
            continue
filtered_ids = sorted(list(set(filtered_ids)))
filtered_ids

[11, 14, 440, 1608, 3386]